# **DATA EXTRACTION**

## **Import Libraries & MIMIC-III Data Files**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import Counter
from itertools import chain
import pickle

import pandas as pd
from ipywidgets.widgets import HTML, Output, HBox, Button, Text
from IPython.display import display, clear_output
import json

import sys
sys.path.append('/content/BHC-Summarisation')

from mimic_note_clean import parse_radiology_report_text, parse_echo_report_text, parse_nurs_prog_notes_text, \
    parse_phys_prog_notes_text, parse_phys_intens_notes_text, parse_phys_attend_notes_text, parse_phys_res_attnd_adm_notes_text, \
    parse_phys_res_attn_micu_notes_text, parse_phys_res_attn_prog_notes_text, parse_phys_surgical_adm_notes_text, \
    parse_respiratory_care_shift_note
from section_parser import section_discharge_texts, dis_sum_brief_hos_course_headers

In [ ]:
notes = pd.read_csv("/content/drive/MyDrive/MIMIC-III/noteevents.csv")

In [ ]:
admissions = pd.read_csv(r'/content/drive/MyDrive/MIMIC-III/ADMISSIONS.csv')

In [ ]:
services = pd.read_csv(r'/content/drive/MyDrive/MIMIC-III/SERVICES.csv')

In [ ]:
patients = pd.read_csv("/content/drive/MyDrive/MIMIC-III/PATIENTS.csv")

In [ ]:
precriptions = pd.read_csv("/content/drive/MyDrive/MIMIC-III/PRESCRIPTIONS.csv")

<ipython-input-8-997d87bf45db>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  precriptions = pd.read_csv("/content/drive/MyDrive/MIMIC-III/PRESCRIPTIONS.csv")


##**Straight-Forward Extraction**

In [ ]:
admissions_columns = ["SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", "DIAGNOSIS"]
admissions_extracted = admissions[admissions_columns]

services_columns = ["SUBJECT_ID", "HADM_ID", "CURR_SERVICE"]
services_extracted = services[services_columns]

patients_columns = ["SUBJECT_ID",  "GENDER", "DOB"]
patients_extracted = patients[patients_columns]

precriptions_columns = [ 'SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG']
precriptions_extracted = precriptions[precriptions_columns]

data = pd.merge(admissions_extracted, services_extracted, on=["SUBJECT_ID", "HADM_ID"], how="inner")
temp = pd.merge(patients_extracted,precriptions_extracted, on=["SUBJECT_ID"], how="inner")
data = pd.merge(data,temp, on=["SUBJECT_ID", "HADM_ID"], how="inner")

In [ ]:
data['ADMITTIME'] = pd.to_datetime(data['ADMITTIME'])
data['DISCHTIME'] = pd.to_datetime(data['DISCHTIME'])
data['STARTDATE'] = pd.to_datetime(data['STARTDATE'])
data['ENDDATE'] = pd.to_datetime(data['ENDDATE'])

##**Enriching data with diagonosis and procedures data**

In [ ]:
d_path = '/content/drive/MyDrive/MIMIC-III/DIAGNOSES_ICD.csv'
dcode_path = '/content/drive/MyDrive/MIMIC-III/D_ICD_DIAGNOSES.csv'

In [ ]:
p_path = '/content/drive/MyDrive/MIMIC-III/PROCEDURES_ICD.csv'
pcode_path = '/content/drive/MyDrive/MIMIC-III/D_ICD_PROCEDURES.csv'

In [ ]:
diagnosis = pd.read_csv(d_path)
diagnosis_codes = pd.read_csv(dcode_path)

In [ ]:
procedures = pd.read_csv(p_path)
procedures_codes = pd.read_csv(pcode_path)

In [ ]:
diagnosis.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [ ]:
diagnosis_codes.columns

Index(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE'], dtype='object')

In [ ]:
procedures.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [ ]:
procedures_codes.columns

Index(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE'], dtype='object')

In [ ]:
diagnosis = diagnosis[ ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
diagnosis_codes = diagnosis_codes[['ICD9_CODE', 'LONG_TITLE']]

In [ ]:
procedures = procedures[ ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
procedures_codes = procedures_codes[['ICD9_CODE', 'LONG_TITLE']]

In [ ]:
diagnosis_ = pd.merge(diagnosis, diagnosis_codes, on=["ICD9_CODE"], how="inner")
procedures_ = pd.merge(procedures, procedures_codes, on=["ICD9_CODE"], how="inner")

In [ ]:
diagnosis_.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'LONG_TITLE'], dtype='object')

In [ ]:
procedures_.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'LONG_TITLE'], dtype='object')

In [ ]:
diagnosis_enriched = diagnosis_.groupby(['SUBJECT_ID', 'HADM_ID']).agg({'LONG_TITLE': lambda x: list(x)}).reset_index()

In [ ]:
diagnosis_enriched.columns

Index(['SUBJECT_ID', 'HADM_ID', 'LONG_TITLE'], dtype='object')

In [ ]:
diagnosis_enriched.rename(columns={'LONG_TITLE': 'DIAGNOSIS_TITLES'}, inplace=True)

In [ ]:
procedures_enriched = procedures_.groupby(['SUBJECT_ID', 'HADM_ID']).agg({'LONG_TITLE': lambda x: list(x)}).reset_index()

In [ ]:
procedures_enriched.rename(columns={'LONG_TITLE': 'PROCEDURE_TITLES'}, inplace=True)

In [ ]:
procedures_enriched.columns

Index(['SUBJECT_ID', 'HADM_ID', 'PROCEDURE_TITLES'], dtype='object')

In [ ]:
data = pd.merge(data, diagnosis_enriched, on=["SUBJECT_ID", "HADM_ID"], how="inner")
data = pd.merge(data, procedures_enriched, on=["SUBJECT_ID", "HADM_ID"], how="inner")

In [ ]:
data.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS',
       'CURR_SERVICE', 'GENDER', 'DOB', 'STARTDATE', 'ENDDATE', 'DRUG',
       'DIAGNOSIS_TITLES', 'PROCEDURE_TITLES'],
      dtype='object')

In [ ]:
duplicate = notes

##**Handling Clinical Notes**

###**Organize Data**

In [ ]:
notes = duplicate.iloc[:1000]

In [ ]:
notes.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE',
       'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR',
       'TEXT'],
      dtype='object')

In [ ]:
columns_to_drop = ['ROW_ID', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR']
notes.drop(columns=columns_to_drop, inplace=True)

<ipython-input-35-514a5782e04c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes.drop(columns=columns_to_drop, inplace=True)


In [ ]:
notes['CHARTDATE'] = pd.to_datetime(notes['CHARTDATE'])
notes['DOB'] = pd.to_datetime(notes['DOB'])

In [ ]:
patients_col = ["DOB","SUBJECT_ID"]
patients_extract = patients[patients_columns]
notes = pd.merge(notes, patients_extract, on=["SUBJECT_ID"], how="inner")

In [ ]:
notes.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'TEXT', 'GENDER',
       'DOB'],
      dtype='object')

In [ ]:
notes['SUBJECT_AGE'] = notes.apply(lambda r: r.CHARTDATE.year - r.DOB.year, axis=1)

In [ ]:
notes = notes[notes['SUBJECT_AGE'] <= 100]

In [ ]:
columns_to_drop = ['DOB','CHARTDATE']
notes.drop(columns=columns_to_drop, inplace=True)

<ipython-input-48-8697478f7fd9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes.drop(columns=columns_to_drop, inplace=True)


###**Preprocessing on Progress notes**

> Regex - Pattern Matching

> String Manipulation

> Conditional Logic for data filtering

> Medical Codes to Clinical terms

#### **Hospital-course-content**

In [ ]:
pat = [
    '(brief hospital course:)',
    '(hospital course:)',
    '(concise summary of hospital course by issue/system:)',
    '(summary of hospital course by systems:)',
    '(hospital course by systems including pertinent laboratory\ndata:)',
    '(details of hospital course:)',
]
contains_brief_hosp_course_sec = notes.TEXT.str.lower().str.contains('|'.join(pat))
brief_hos_course_notes = notes[contains_brief_hosp_course_sec]

<ipython-input-49-6e8f07d7239c>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_brief_hosp_course_sec = notes.TEXT.str.lower().str.contains('|'.join(pat))


In [ ]:
comp_pat = re.compile('|'.join(pat), re.IGNORECASE)

In [ ]:
matches = notes.TEXT.apply(lambda t: re.search(comp_pat, t))

In [ ]:
dis_notes = notes.loc[~pd.isna(matches)]

In [ ]:
matches = matches.loc[~pd.isna(matches)]

In [ ]:
dis_notes['match_start'] = matches

<ipython-input-54-bada1a023e63>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_notes['match_start'] = matches


In [ ]:
dis_notes['HOS_COURSE'] = dis_notes.apply(lambda r: r.TEXT[r.match_start.start():], axis=1)

<ipython-input-55-84cec3790197>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_notes['HOS_COURSE'] = dis_notes.apply(lambda r: r.TEXT[r.match_start.start():], axis=1)


In [ ]:
samples = dis_notes.sample(100)
output_file = 'dis_notes_sample.csv'
samples.to_csv(output_file, index=False)

In [ ]:
dis_notes['next_sec_matches'] = [re.search('(\n\n(?!#|\*|\d|\s|\[|\().*:)', s, re.IGNORECASE) for s in dis_notes.HOS_COURSE]

<ipython-input-57-f777e8a4570a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_notes['next_sec_matches'] = [re.search('(\n\n(?!#|\*|\d|\s|\[|\().*:)', s, re.IGNORECASE) for s in dis_notes.HOS_COURSE]


####**Sub-section Extraction**

In [ ]:
def extract_subsections(text):
  regex_pattern = r'^([A-Z][A-Z ]+):'
  subtopic_matches = re.findall(regex_pattern, text, re.MULTILINE)
  subtopics = [match.strip() for match in subtopic_matches]
  return subtopics

In [ ]:
dis_notes['HOS_COURSE_SUBSECTIONS'] = notes['TEXT'].apply(lambda x: extract_subsections(x))

<ipython-input-59-4337812ae67f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_notes['HOS_COURSE_SUBSECTIONS'] = notes['TEXT'].apply(lambda x: extract_subsections(x))


In [ ]:
dis_notes

,Unnamed: 0,SUBJECT_ID,HADM_ID,TEXT,GENDER,SUBJECT_AGE,match_start,HOS_COURSE,next_sec_matches,HOS_COURSE_SUBSECTIONS
1,245,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...,F,87,"<re.Match object; span=(4964, 4980), match='HO...",HOSPITAL COURSE: Infectious disease - As note...,"<re.Match object; span=(8748, 8773), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
2,1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...,F,81,"<re.Match object; span=(5537, 5553), match='HO...",HOSPITAL COURSE:\n1. COPD/dyspnea/pneumonia: ...,"<re.Match object; span=(4827, 4849), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
3,2,13702,167118.0,Admission Date: [**2119-5-4**] D...,F,82,"<re.Match object; span=(3609, 3631), match='Br...",Brief Hospital Course:\n82 y/o female admitted...,"<re.Match object; span=(3709, 3736), match='\n...",[PMH]
4,3,13702,196489.0,Admission Date: [**2124-7-21**] ...,F,87,"<re.Match object; span=(6373, 6395), match='Br...","Brief Hospital Course:\n87 yo F with h/o CHF, ...","<re.Match object; span=(5073, 5100), match='\n...","[HEENT, CV, GU, IMPRESSION, LABS AT DISCHARGE]"
5,4,26880,135453.0,Admission Date: [**2162-3-3**] D...,M,82,"<re.Match object; span=(10762, 10784), match='...",Brief Hospital Course:\nMr. [**Known lastname ...,"<re.Match object; span=(349, 368), match='\n\n...","[VS, CV, IMPRESSION, IMPRESSION, CT ABDOMEN WI..."
...,...,...,...,...,...,...,...,...,...,...
993,993,12411,173718.0,Admission Date: [**2183-3-12**] Discharge...,F,42,"<re.Match object; span=(4379, 4395), match='HO...",HOSPITAL COURSE: While in the Emergency Depar...,"<re.Match object; span=(2284, 2342), match='\n...","[CHIEF COMPLAINT, HISTORY OF PRESENT ILLNESS, ..."
994,994,12411,163752.0,Admission Date: [**2184-7-5**] Discharge ...,F,43,"<re.Match object; span=(3372, 3388), match='HO...",HOSPITAL COURSE: The patient was admitted to ...,"<re.Match object; span=(2546, 2573), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
995,995,15198,137838.0,Admission Date: [**2184-10-14**] Discha...,F,76,"<re.Match object; span=(2671, 2687), match='HO...",HOSPITAL COURSE: On the day of admission pati...,"<re.Match object; span=(3071, 3095), match='\n...","[CHIEF COMPLAINT, HISTORY OF PRESENT ILLNESS, ..."
998,998,18511,123903.0,Admission Date: [**2191-2-23**] ...,M,67,"<re.Match object; span=(3180, 3202), match='Br...",Brief Hospital Course:\nPresented to emergency...,"<re.Match object; span=(1019, 1046), match='\n...",[PMH]


In [ ]:
dis_notes = dis_notes[dis_notes['HOS_COURSE_SUBSECTIONS'].apply(len) > 0]

In [ ]:
dis_notes.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'TEXT', 'GENDER', 'SUBJECT_AGE',
       'match_start', 'HOS_COURSE', 'next_sec_matches',
       'HOS_COURSE_SUBSECTIONS'],
      dtype='object')

In [ ]:
columns_to_drop = ['Unnamed: 0',
       'match_start', ]
dis_notes.drop(columns=columns_to_drop, inplace=True)

<ipython-input-63-e1ffb1059864>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_notes.drop(columns=columns_to_drop, inplace=True)


####**Visualize and Duplicates Removal**

In [ ]:
samples = dis_notes.sample(100)

In [ ]:
full_sec = samples.iloc[:100].HOS_COURSE
sec = samples.iloc[:100].HOS_COURSE_SUBSECTIONS
HTML(value=f'<div style="width=100%">' +\
     f'<div style="display:inline-block;width:50%; white-space: pre-wrap; line-height: 1.2">{full_sec}</div>' +\
     f'<div style="display:inline-block;width:50%; white-space: pre-wrap; line-height: 1.2">{sec}</div>' +\
     '</div>')

HTML(value='<div style="width=100%"><div style="display:inline-block;width:50%; white-space: pre-wrap; line-he…

In [ ]:
dis_notes = dis_notes[~dis_notes.duplicated('HOS_COURSE_SUBSECTIONS', keep=False)]

In [ ]:
dis_notes.reset_index(drop=True).to_csv('dis_notes_hosp_course_extracted.csv')

In [ ]:
dis_notes

,SUBJECT_ID,HADM_ID,TEXT,GENDER,SUBJECT_AGE,HOS_COURSE,next_sec_matches,HOS_COURSE_SUBSECTIONS
1,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...,F,87,HOSPITAL COURSE: Infectious disease - As note...,"<re.Match object; span=(8748, 8773), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
2,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...,F,81,HOSPITAL COURSE:\n1. COPD/dyspnea/pneumonia: ...,"<re.Match object; span=(4827, 4849), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
4,13702,196489.0,Admission Date: [**2124-7-21**] ...,F,87,"Brief Hospital Course:\n87 yo F with h/o CHF, ...","<re.Match object; span=(5073, 5100), match='\n...","[HEENT, CV, GU, IMPRESSION, LABS AT DISCHARGE]"
5,26880,135453.0,Admission Date: [**2162-3-3**] D...,M,82,Brief Hospital Course:\nMr. [**Known lastname ...,"<re.Match object; span=(349, 368), match='\n\n...","[VS, CV, IMPRESSION, IMPRESSION, CT ABDOMEN WI..."
9,56174,163469.0,Admission Date: [**2118-8-10**] ...,F,45,Brief Hospital Course:\nPost-operatively she w...,"<re.Match object; span=(575, 602), match='\n\n...","[DIAGNOSIS, FINDINGS, IMPRESSION, FOLLOWING]"
...,...,...,...,...,...,...,...,...
990,10830,120142.0,Admission Date: [**2116-7-6**] D...,F,86,Brief Hospital Course:\n85 y/o F with MDS/RAEB...,"<re.Match object; span=(3230, 3257), match='\n...","[ROS, HEENT, CV, LUNGS, ABD, EXT, ECG]"
993,12411,173718.0,Admission Date: [**2183-3-12**] Discharge...,F,42,HOSPITAL COURSE: While in the Emergency Depar...,"<re.Match object; span=(2284, 2342), match='\n...","[CHIEF COMPLAINT, HISTORY OF PRESENT ILLNESS, ..."
994,12411,163752.0,Admission Date: [**2184-7-5**] Discharge ...,F,43,HOSPITAL COURSE: The patient was admitted to ...,"<re.Match object; span=(2546, 2573), match='\n...","[HISTORY OF PRESENT ILLNESS, PAST MEDICAL HIST..."
995,15198,137838.0,Admission Date: [**2184-10-14**] Discha...,F,76,HOSPITAL COURSE: On the day of admission pati...,"<re.Match object; span=(3071, 3095), match='\n...","[CHIEF COMPLAINT, HISTORY OF PRESENT ILLNESS, ..."


In [ ]:
dis_notes_ = dis_notes

####**KEY SECTIONS**

In [ ]:
out_texts = dis_notes.TEXT.apply(section_discharge_texts)

In [ ]:
pickle.dump(out_texts, open('dis_course_secs.pickle', 'wb'))

In [ ]:
brief_hos_course_secs = []
other_secs = []
dis_course_headers = [h.lower().strip().replace(':', '') for h in dis_sum_brief_hos_course_headers]
for row in out_texts:
    brief_hos_course = None
    brief_hos_course_idx = None
    for i, (sec, sec_name) in enumerate(zip(row[0], row[1])):
        if sec_name in dis_course_headers:
            brief_hos_course = sec
            brief_hos_course_idx = i
    if brief_hos_course_idx is not None:
        row_secs = row[0][0:brief_hos_course_idx] + row[0][brief_hos_course_idx+1:]
        row_sec_headers = row[1][0:brief_hos_course_idx] + row[1][brief_hos_course_idx+1:]
    brief_hos_course_secs.append(brief_hos_course)
    other_secs.append([row_secs, row_sec_headers])

In [ ]:
other_sec_header_counts = Counter(list(chain.from_iterable([o[1] for o in other_secs])))

In [ ]:
sec_headers_df = pd.Series(other_sec_header_counts).sort_values(ascending=False).reset_index()
print(sec_headers_df.shape)
hdrs = sec_headers_df['index'].tolist()

(60, 2)


In [ ]:
print('\n'.join([f'{h}:{c}' for h,c in zip(sec_headers_df['index'].tolist(), sec_headers_df[0].tolist())]))

preamble:685
history of present illness:666
past medical history:662
discharge medications:563
medications on admission:561
physical exam:540
discharge diagnosis:531
discharge condition:505
discharge instructions:499
discharge disposition:496
dictated by:198
impression:188
physical examination:116
condition on discharge:73
discharge diagnoses:73
laboratory data:72
discharge status:60
medications:53
medications on discharge:39
disposition:27
hpi:22
condition at discharge:22
follow-up:16
final diagnosis:16
follow up:16
cv:11
id:10
discharge instructions/followup:10
brief hospital course:8
transitional issues:8
gi:7
procedure:6
diagnosis:5
labs on discharge:5
discharge followup:5
fen:4
discharge plan:4
physical examination on discharge:4
plan:4
hospital course:4
note:3
addendum:3
final diagnoses:3
follow-up instructions:3
code:2
active issues:2
code status:2
postoperative course was remarkable for the following:2
final discharge diagnoses:2
comm:2
chronic issues:1
condition:1
condition of

### **Segmenting and Structuring Discharge Summary Content**

In [ ]:
headers_to_use = [
    "preamble",
    "history of present illness",
    "past medical history",
    "discharge medications",
    "medications on admission",
    "discharge diagnosis",
    "discharge disposition",
    "impression",
    "problem list",
]

In [ ]:
filtered_out_texts = []
for secs, sec_headers, _ in out_texts:
    filtered_secs, filtered_headers = [], []
    for sec, sec_h in zip(secs, sec_headers):
        if sec_h in headers_to_use:
            filtered_secs.append(sec)
            filtered_headers.append(sec_h)
    filtered_out_texts.append([filtered_secs, filtered_headers])

In [ ]:
dis_notes.columns

Index(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'GENDER', 'SUBJECT_AGE', 'HOS_COURSE',
       'next_sec_matches', 'HOS_COURSE_SUBSECTIONS'],
      dtype='object')

In [ ]:
dis_notes['other_dis_notes_secs'] = filtered_out_texts

In [ ]:
def extract_subsection_content(text, target_subsection, subsections):
    """
    Extracts the content of the target_subsection from the given text using the subsections list.

    Parameters:
        text (str): The input discharge summary text.
        target_subsection (str): The name of the subsection whose content is to be extracted.
        subsections (list): List of all subsections in the discharge summary.

    Returns:
        str: The content of the target_subsection if found, else an empty string.
    """
    subsections_lower = [subsection.lower() for subsection in subsections]
    if target_subsection.lower() in subsections_lower:
        start_pattern = target_subsection.lower()
        start_position = text.lower().find(start_pattern)
    else:
        return ""

    if start_position is not None:
        end_positions = []
        for subsection in subsections_lower:
            if subsection != start_pattern:
                end_position = text.lower().find(subsection)
                if end_position != -1:
                    end_positions.append(end_position)
        valid_end_positions = [end_pos for end_pos in end_positions if end_pos > start_position]
        if valid_end_positions:
            end_position = min(valid_end_positions)
        else:
            end_position = len(text)
        content = text[start_position:end_position].strip()
        return content
    return ""
subsections = ['FINAL DIAGNOSES', 'FOLLOW-UP PLANS', 'DISCHARGE MEDICATIONS', 'DISCHARGE STATUS', 'DISCHARGE CONDITION',
               'BRIEF SUMMARY OF HOSPITAL COURSE', 'LABORATORY STUDIES', 'PHYSICAL EXAM AT TIME OF ADMISSION', 'SOCIAL HISTORY',
               'FAMILY HISTOR', 'ALLERGIES', 'MEDICATIONS ON ADMISSION', 'PAST MEDICAL HISTORY', 'HISTORY OF PRESENT ILLNESS',
               'Admission Date', 'Discharge Date', 'Date of Birth', 'Sex', 'Service']

In [ ]:
targets = ['FOLLOW-UP PLANS', 'DISCHARGE CONDITION', 'PAST MEDICAL HISTORY']
for target in targets:
    dis_notes[target] = dis_notes['TEXT'].apply(lambda text: extract_subsection_content(text, target, subsections))

In [ ]:
dis_notes.columns

Index(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'GENDER', 'SUBJECT_AGE', 'HOS_COURSE',
       'next_sec_matches', 'HOS_COURSE_SUBSECTIONS', 'other_dis_notes_secs',
       'FOLLOW-UP PLANS', 'DISCHARGE CONDITION', 'PAST MEDICAL HISTORY'],
      dtype='object')

## **Structuring and Integration**

In [ ]:
hadms_to_dis_course = [{'HADM_ID': r.HADM_ID, 'text': r.HOS_COURSE_SUBSECTIONS} for r in dis_notes.itertuples() if len(r.HOS_COURSE_SUBSECTIONS) > 0]

In [ ]:
hadms_to_hosp_course = {}
for item in hadms_to_dis_course:
    h = item['HADM_ID']
    h_notes = dis_notes[dis_notes.HADM_ID == h]
    secs, sec_headers = dis_notes[dis_notes.HADM_ID == h].iloc[0].other_dis_notes_secs
    dis_notes_extension = '\n\n'.join([f'{h}\n{sec}' for sec, h in zip(secs, sec_headers)])
    hadms_to_hosp_course[h] = f'\nDischarge summary\n{dis_notes_extension}'

# **Save the Data for easy retrieval**

In [ ]:
json.dump(hadms_to_dis_course, open('hadms_to_dis_course.json', 'w'))

In [ ]:
json.dump(hadms_to_hosp_course, open('hadms_to_hosp_course.json', 'w'))

In [ ]:
data = data.merge(dis_notes, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [ ]:
data.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS',
       'CURR_SERVICE', 'GENDER', 'DOB', 'STARTDATE', 'ENDDATE', 'DRUG',
       'DIAGNOSIS_TITLES', 'PROCEDURE_TITLES', 'ALLERGIES', 'FINAL DIAGNOSES',
       'DISCHARGE MEDICATIONS', 'TEXT', 'SUBJECT_AGE', 'HOS_COURSE',
       'HOS_COURSE_SUBSECTIONS', 'DIS_NOTES', 'FOLLOW-UP PLANS',
       'DISCHARGE CONDITION', 'PAST MEDICAL HISTORY'],
      dtype='object')

In [ ]:
csv_data = data.to_csv(index=False)
output_file = '/content/drive/MyDrive/Data.csv'
with open(output_file, 'w') as f:
    f.write(csv_data)
from IPython.display import FileLink
FileLink(output_file)


/content/drive/MyDrive/Data.csv

In [ ]:
import pandas as pd